# LangChain 모델

LLM(대규모 언어 모델)은 사람처럼 텍스트를 해석하고 생성할 수 있는 강력한 AI 도구입니다. 각 작업에 대한 전문적인 훈련 없이도 콘텐츠 작성, 언어 번역, 요약 및 질문 응답에 사용할 수 있습니다.

텍스트 생성 외에도 많은 모델이 다음을 지원합니다:

- 도구 호출 - 외부 도구(데이터베이스 쿼리 또는 API 호출 등)를 호출하고 결과를 응답에 사용
- 구조화된 출력 - 모델의 응답이 정의된 형식을 따르도록 제한
- 멀티모달리티 - 이미지, 오디오, 비디오 등 텍스트가 아닌 데이터 처리 및 반환
- 추론 - 결론에 도달하기 위한 다단계 추론 수행

모델은 에이전트의 추론 엔진입니다. 에이전트의 의사 결정 프로세스를 주도하여 어떤 도구를 호출할지, 결과를 해석하는 방법, 최종 답변을 제공할 시기를 결정합니다.

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## 기본 사용법

모델은 두 가지 방식으로 활용할 수 있습니다:

1. **에이전트와 함께** - 에이전트를 생성할 때 모델을 동적으로 지정
2. **독립 실행형** - 에이전트 프레임워크 없이 텍스트 생성, 분류 또는 추출 작업을 위해 모델을 직접 호출

동일한 모델 인터페이스가 두 컨텍스트에서 모두 작동하므로 간단하게 시작하여 필요에 따라 더 복잡한 에이전트 기반 워크플로로 확장할 수 있습니다.

### 모델 초기화

LangChain에서 독립 실행형 모델을 시작하는 가장 쉬운 방법은 `init_chat_model`을 사용하여 선택한 채팅 모델 제공자로부터 모델을 초기화하는 것입니다.

#### OpenAI

In [ ]:
import os
from langchain.chat_models import init_chat_model

# init_chat_model 사용
model = init_chat_model("openai:gpt-4.1-mini")

In [ ]:
# 또는 모델 클래스 직접 사용
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

#### Anthropic

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("anthropic:claude-sonnet-4-5")

In [ ]:
# 또는 모델 클래스 직접 사용
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-sonnet-4-5")

#### Google Gemini

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

### 모델 호출

In [ ]:
response = model.invoke("Why do parrots talk?")
print(response)

### 주요 메서드

1. **Invoke** - 모델이 메시지를 입력으로 받아 완전한 응답을 생성한 후 메시지를 출력합니다.
2. **Stream** - 모델을 호출하지만 실시간으로 생성되는 출력을 스트리밍합니다.
3. **Batch** - 더 효율적인 처리를 위해 여러 요청을 일괄로 모델에 전송합니다.

## 매개변수

채팅 모델은 동작을 구성하는 데 사용할 수 있는 매개변수를 받습니다. 지원되는 전체 매개변수 세트는 모델 및 제공자에 따라 다르지만 표준 매개변수는 다음과 같습니다:

- `model`: 사용하려는 특정 모델의 이름 또는 식별자
- `api_key`: 모델 제공자와 인증하는 데 필요한 키
- `temperature`: 모델 출력의 무작위성 제어 (높을수록 더 창의적)
- `timeout`: 요청을 취소하기 전에 모델 응답을 기다리는 최대 시간(초)
- `max_tokens`: 응답의 총 토큰 수 제한
- `max_retries`: 실패 시 요청을 재전송하는 최대 시도 횟수

In [ ]:
# 매개변수를 사용하여 모델 초기화
model = init_chat_model(
    "anthropic:claude-sonnet-4-5",
    temperature=0.7,    # 응답의 창의성 제어
    timeout=30,         # 요청 타임아웃
    max_tokens=1000,    # 최대 생성 토큰 수
)

## 호출 방법

### Invoke

모델을 호출하는 가장 간단한 방법은 단일 메시지 또는 메시지 목록과 함께 `invoke()`를 사용하는 것입니다.

In [ ]:
# 단일 메시지
response = model.invoke("Why do parrots have colorful feathers?")
print(response)

In [ ]:
# 대화 기록을 나타내는 메시지 목록
conversation = [
    {"role": "system", "content": "You are a helpful assistant that translates English to French."},
    {"role": "user", "content": "Translate: I love programming."},
    {"role": "assistant", "content": "J'adore la programmation."},
    {"role": "user", "content": "Translate: I love building applications."}
]

response = model.invoke(conversation)
print(response)

In [ ]:
# 메시지 객체 사용
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = model.invoke(conversation)
print(response)

### Stream

대부분의 모델은 생성되는 동안 출력 콘텐츠를 스트리밍할 수 있습니다. 출력을 점진적으로 표시함으로써 스트리밍은 특히 긴 응답에 대해 사용자 경험을 크게 향상시킵니다.

In [ ]:
# 기본 텍스트 스트리밍
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.content, end="", flush=True)

In [ ]:
# 전체 메시지를 구성하기 위해 청크 누적
full = None
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.content)

# 결과 메시지는 invoke()로 생성된 메시지와 동일하게 처리될 수 있습니다
print("\n최종 메시지:")
print(full)

### Batch

독립적인 요청 모음을 일괄 처리하면 처리가 병렬로 수행될 수 있으므로 성능이 크게 향상되고 비용이 절감됩니다.

In [ ]:
# 여러 요청을 일괄 처리
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])

for response in responses:
    print(response)
    print("---")

In [ ]:
# 완료 시 일괄 응답 산출
for response in model.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)
    print("---")

## 도구 호출

모델은 데이터베이스에서 데이터를 가져오거나, 웹을 검색하거나, 코드를 실행하는 등의 작업을 수행하는 도구를 호출하도록 요청할 수 있습니다.

정의한 도구를 모델에서 사용할 수 있도록 하려면 `bind_tools()`를 사용하여 바인딩해야 합니다.

In [ ]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."

# 도구를 모델에 바인딩
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

# 모델이 수행한 도구 호출 확인
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

### 도구 실행 루프

모델이 도구 호출을 반환하면 도구를 실행하고 결과를 모델에 다시 전달해야 합니다. 이렇게 하면 모델이 도구 결과를 사용하여 최종 응답을 생성할 수 있는 대화 루프가 생성됩니다.

In [ ]:
# 도구를 모델에 바인딩
model_with_tools = model.bind_tools([get_weather])

# 1단계: 모델이 도구 호출 생성
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# 2단계: 도구 실행 및 결과 수집
for tool_call in ai_msg.tool_calls:
    # 생성된 인수로 도구 실행
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# 3단계: 최종 응답을 위해 결과를 모델에 다시 전달
final_response = model_with_tools.invoke(messages)
print(final_response.content)

### 병렬 도구 호출

많은 모델이 적절한 경우 여러 도구를 병렬로 호출하는 것을 지원합니다. 이를 통해 모델은 서로 다른 소스에서 동시에 정보를 수집할 수 있습니다.

In [ ]:
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(
    "What's the weather in Boston and Tokyo?"
)

# 모델이 여러 도구 호출을 생성할 수 있습니다
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_weather', 'args': {'location': 'Tokyo'}, 'id': 'call_2'},
# ]

# 모든 도구 실행 (비동기로 병렬 처리 가능)
results = []
for tool_call in response.tool_calls:
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
        results.append(result)

print(results)

## 구조화된 출력

모델이 주어진 스키마와 일치하는 형식으로 응답을 제공하도록 요청할 수 있습니다. 이는 출력을 쉽게 구문 분석하고 후속 처리에 사용할 수 있도록 하는 데 유용합니다.

### Pydantic 모델

In [ ]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)
# Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

### TypedDict

In [ ]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]

model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)
# {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}

### 중첩된 구조

In [ ]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke(
    "Provide detailed information about the movie Inception including cast and genres"
)
print(response)

## 고급 주제

### 멀티모달

특정 모델은 이미지, 오디오, 비디오 등 텍스트가 아닌 데이터를 처리하고 반환할 수 있습니다. 콘텐츠 블록을 제공하여 텍스트가 아닌 데이터를 모델에 전달할 수 있습니다.

In [ ]:
# 멀티모달 입력 예시 (이미지 URL 포함)
message = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
        ]
    }
]

# response = model.invoke(message)
# print(response.content)

### 추론

최신 모델은 결론에 도달하기 위해 다단계 추론을 수행할 수 있습니다. 기본 모델에서 지원되는 경우 이 추론 프로세스를 표시하여 모델이 최종 답변에 도달한 방법을 더 잘 이해할 수 있습니다.

In [ ]:
# 추론 출력 스트리밍
for chunk in model.stream("Why do parrots have colorful feathers?"):
    reasoning_steps = [r for r in chunk.content_blocks if r.get("type") == "reasoning"]
    if reasoning_steps:
        print(reasoning_steps)
    else:
        print(chunk.content)

### 토큰 사용량

많은 모델 제공자가 호출 응답의 일부로 토큰 사용량 정보를 반환합니다. 사용 가능한 경우 이 정보는 해당 모델이 생성한 AIMessage 객체에 포함됩니다.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

model_1 = init_chat_model(model="openai:gpt-4.1-mini")
model_2 = init_chat_model(model="anthropic:claude-3-5-haiku-latest")

# 콜백 핸들러를 사용하여 토큰 사용량 추적
callback = UsageMetadataCallbackHandler()
result_1 = model_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = model_2.invoke("Hello", config={"callbacks": [callback]})

print(callback.usage_metadata)

### 호출 구성

모델을 호출할 때 `config` 매개변수를 통해 추가 구성을 전달할 수 있습니다. 이를 통해 실행 동작, 콜백 및 메타데이터 추적을 런타임에 제어할 수 있습니다.

In [ ]:
# 구성을 사용한 호출
response = model.invoke(
    "Tell me a joke",
    config={
        "run_name": "joke_generation",      # 이 실행의 커스텀 이름
        "tags": ["humor", "demo"],          # 분류를 위한 태그
        "metadata": {"user_id": "123"},     # 커스텀 메타데이터
    }
)

print(response.content)

### 구성 가능한 모델

`configurable_fields`를 지정하여 런타임 구성 가능한 모델을 만들 수 있습니다. 모델 값을 지정하지 않으면 `model` 및 `model_provider`가 기본적으로 구성 가능합니다.

In [ ]:
from langchain.chat_models import init_chat_model

# 구성 가능한 모델 생성
configurable_model = init_chat_model(temperature=0)

# 다른 모델로 실행
response1 = configurable_model.invoke(
    "what's your name",
    config={"configurable": {"model": "gpt-4.1-mini"}},
)
print("gpt-4.1-mini:", response1.content)

response2 = configurable_model.invoke(
    "what's your name",
    config={"configurable": {"model": "claude-sonnet-4-5"}},
)
print("Claude:", response2.content)